# Exercise 04 - Due Friday, October 21 at 12pm

*Objectives*: Gain experience loading a CSV dataset into a database model you define yourself and using SQL to explore its contents. Explore the data by writing and executing a number of SQL queries using common syntax and functions and describing your findings.

*Grading criteria*: The tasks should all be completed, and questions should all be answered with SQL queries in the space provided, unless a text answer is requested. Results should be correct, according to the embedded tests. The notebook itself should be completely reproducible; from start to finish, another person should be able to use the same code to obtain the same results as yours.  Note that you will receive no more than partial credit if you do not add text/markdown cells explaining your thinking at each major step in each problem.

*Deadline*: Friday, October 21, 12pm. Submit your notebook to Blackboard and push it to your GitHub repository.

## Setup - obtain data and create database

The US City Open Data Census has data on [restaurant inspections from all over the country](http://us-city.census.okfn.org/dataset/food-safety).  Let's take a look at data from Albuquerque.  *Note*: metadata and other details for this dataset are available at http://data.cabq.gov/business/foodinspections/.

Fyi, to save you a step, I've converted this file into a clean CSV file by using the `dos2unix` command to change its line endings and to switch it to use commas instead of tabs using `csvformat -t inputfile.csv > outputfile.csv`.

First we download the dataset:

In [1]:
!wget https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/exercises/abq-food-inspections-fy2010.csv

--2016-10-24 00:15:10--  https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/exercises/abq-food-inspections-fy2010.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.32.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.32.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5025226 (4.8M) [text/plain]
Saving to: ‘abq-food-inspections-fy2010.csv’

abq-food-inspection 100%[=====================>]   4.79M  --.-KB/s   in 0.09s  

2016-10-24 00:15:11 (52.4 MB/s) - ‘abq-food-inspections-fy2010.csv’ saved [5025226/5025226]



## Problem 1 - examine the data (35 points)

Use `csvstat` and other tools as you see fit and observe its contents.  Review the metadata definitions linked to above.  Describe what you see:  What do all the columns mean?  Are there null values?  Which columns interest you the most?  Which columns present some complications or questions you would like to ask?

Insert a combination of code and text/markdown cells here to review the data and describe what you find.

Rename csv file to make it easier to work with

In [2]:
!mv abq-food-inspections-fy2010.csv inspections.csv

Take a look at the columns using csvstat

In [3]:
!csvstat inspections.csv

  1. FACILITY_NAME
	<class 'str'>
	Nulls: False
	Unique values: 2615
	5 most frequent values:
		SMITHS FOOD AND DRUG CENTERS INC:	154
		ISOTOPES PARK - OVATIONS:	93
		PER EHSII CECELIA GARCIA OOB LA REYNA DEL SUR:	85
		ISLETA AMPHITHEATER:	68
		ECHO INC:	68
	Max length: 77
  2. FACILITY_KEY
	<class 'int'>
	Nulls: False
	Min: 13
	Max: 101482
	Sum: 820405773
	Mean: 46730.79135338346
	Median: 50866.0
	Standard Deviation: 32659.794578837962
	Unique values: 2835
	5 most frequent values:
		46323:	93
		65863:	85
		80949:	68
		32817:	68
		91825:	66
  3. SITE_ADDRESS
	<class 'str'>
	Nulls: True
	Unique values: 2456
	5 most frequent values:
		1601 AVENIDA CESAR CHAVEZ SE:	94
		644 OLD COORS BLVD SW:	85
		8601 CENTRAL AV NE:	77
		2200 SUNPORT BLVD SE:	75
		5210 CENTRAL AV SE:	73
	Max length: 35
  4. CITY
	<class 'str'>
	Nulls: False
	Values: ALBUQUERQUE
  5. STATE
	<class 'str'>
	Nulls: False
	Values: 87, NM
  6. ZIP
	<class 'str'>
	Nulls: False
	Unique values: 48
	5 most frequent values:
		87110

The csvstat command above gives us information about each column in the data file. The file has 27 columns to describe the process of inspecting a restaurant. It has 17,556 rows of data.

The first columns are the facility's name and associated key. 

The site address, city, state, and zip code follow. Of these, only site address contains null values. 

Columns 7 and 8 contain the owner key and the name associate with that key. Column 9 is the nature of the business (such as restaurant or grocery) and it contains some null values. 

Columns 10 through 13 break down the site address into street number, street name, street type (whether it is an avenue, road, etc.), and post-directional (whether the address is specified with NW, SE, or other). All of these columns contain null values.

Column 14 contains phone numbers, which are sometimes null. Columns 15 and 16 are the program category key and its description (type of facility permit). The key column does not contain null values but the description does, which is surprising.

Column 17 is the inspection date. Columns 18 and 19 are the inspection type (key) and its description, such as routine or pre-opening inspections. The key column only contains null values. 

Column 20 is the serial number associate with the inspection. There are a few serial numbers with very high counts, which I am interested in.

Columns 21 and 22 are the action code and its description (what kind of action was taken such as unable to grade, approve, etc.)
Columns 23 and 24 are the result code and its description (the result of the inspection, such as downgrade, in compliance, etc.)
Columns 25 and 26 are the violation code and its description (violations found as a results of inspection, such as none, plumbing, etc.). Both these columns contain null values

Column 27 is the inspection memo, but all its values are null.

## Problem 2 - define a database model and load the data (35 points)

Based on what you found above, create and connect to a new database, define a database table in it, and load this dataset into it.  You may use either of the methods for this step you have seen in earlier class notebooks.  You may choose to eliminate variables/columns if they are not relevant or interesting to you - explain your reasoning if you do.  Either way, you should load a majority of the columns present in the source dataset into the database and all of its rows.

Once your data has loaded successfully, run a `COUNT(*)` query to verify that all the data has loaded correctly.

Insert a combination of code and text/markdown cells here to connect to your database, define your table, load its data, and verify the loaded records.

Restart postgresql

In [4]:
!echo 'redspot' | sudo -S service postgresql restart

[sudo] password for jovyan: Restarting PostgreSQL 9.5 database server: main.


Created the database, activate the sql extension, and connect to database with postgresql

In [5]:
!createdb -U dbuser ex4

In [6]:
%load_ext sql

/opt/conda/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/opt/conda/lib/python3.5/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [7]:
%sql postgresql://dbuser@localhost:5432/ex4

'Connected: dbuser@ex4'

In [8]:
!csvsql --db postgresql://dbuser@localhost:5432/ex4 --insert inspections.csv

Drop both columns that contain only null values

In [9]:
%%sql
SELECT column_name, data_type FROM information_schema.columns
WHERE table_name = 'inspections';

27 rows affected.


column_name,data_type
FACILITY_NAME,character varying
FACILITY_KEY,integer
SITE_ADDRESS,character varying
CITY,character varying
STATE,character varying
ZIP,character varying
OWNER_KEY,integer
OWNER_NAME,character varying
NATURE_OF_BUSINESS,character varying
STREET_NUMBER,character varying


In [10]:
%%sql
ALTER TABLE inspections
DROP COLUMN "INSPECTION_MEMO",
DROP COLUMN "INSPECTION_TYPE";

Done.


[]

Ensure that the data was loaded correctly. There should be 17,556 rows (one less than the csv file because of the lack of header line)

In [11]:
%%sql
SELECT COUNT(*) FROM inspections;

1 rows affected.


count
17556


## Problem 3 - explore your data (30 points)

Now that the data is loaded, ask some questions of it!  Identify key variables of interest and note their ranges along with other useful descriptive statistics.  Choose and define a few lines of inquiry, execute queries for each, and describe what you find in your result sets.  Use any query techniques we've seen in class, including aggregate functions, transformations, subqueries, or others as appropriate. 

If you find interesting patterns, adding plots to your exploration would be useful.

Insert a combination of code and text/markdown cells here to explore your data.

First off, are there any restaurants that get inspected a lot?

In [12]:
%%sql
SELECT "FACILITY_NAME", COUNT("SERIAL_NUM") AS num_inspections FROM inspections
GROUP BY "FACILITY_NAME"
ORDER BY COUNT("SERIAL_NUM") DESC;

2615 rows affected.


FACILITY_NAME,num_inspections
SMITHS FOOD AND DRUG CENTERS INC,154
ISOTOPES PARK - OVATIONS,93
PER EHSII CECELIA GARCIA OOB LA REYNA DEL SUR,85
ECHO INC,68
ISLETA AMPHITHEATER,68
DIONS PIZZA,67
CIRCLE K 8936,66
OOB CALIFORNIA DEL NORTE ICE CREAM LLC,65
TALIN MARKET WORLD FOOD FARE,64
CIRCLE K 8940,61


It appear that one facility in particular has been inspected much more than any other. Let's take a closer look.

In [13]:
%%sql
SELECT "INSPECTION_DESC", COUNT(*) FROM inspections
WHERE "FACILITY_NAME" = 'SMITHS FOOD AND DRUG CENTERS INC'
GROUP BY "INSPECTION_DESC";

4 rows affected.


INSPECTION_DESC,count
ROUTINE FOOD INSPECTION,130
EMERGENCY RESPONSE,15
PHONE CALL,7
ADDITIONAL SERVICE PROVIDED,2


It appears that they have had more "routine" inspections than any other restaurant has had total inspections... Strange. Maybe the restaurant is on an inspector's naughty list

Are there more inspections that take place in certain zip codes?

In [14]:
%%sql
SELECT "ZIP", COUNT(*) AS num_inspections FROM inspections
GROUP BY "ZIP"
ORDER BY COUNT(*) DESC;

48 rows affected.


ZIP,num_inspections
87110,1996
87108,1976
87102,1601
87106,1568
87109,1552
87107,1186
87112,1151
87114,1053
87123,1017
87111,1008


The data is not stored consistently... Sometimes the ZIP is stored as five numbers sometimes it is more detailed. Let's change the query

In [15]:
%%sql
SELECT DISTINCT substring("ZIP",1,5) AS "ZIP", COUNT(*) FROM inspections
GROUP BY substring("ZIP",1,5)
ORDER BY COUNT(*) DESC;

28 rows affected.


ZIP,count
87110,2013
87108,1992
87102,1610
87106,1568
87109,1555
87107,1186
87112,1154
87114,1083
87111,1027
87123,1017


As we can see some zip codes are seeing a lot more activity than others

## Bonus (10 points)

We have seen a few simple techniques for normalizing data in class.  Choose a column or set of columns that might be redundant and normalize them out to another table:

* Identify the columns to normalize in the source data table
* Create one or more new tables to hold the normalized records
* Extract data into the new tables
* Re-insert the new foreign key references from the new table into the source table

I will normalize the columns PROGRAM_CATEGORY and PROGRAM_CATEGORY_DESCRIPTION. To do so, I first create the table to hold the normalized records

In [22]:
%%sql
DROP TABLE IF EXISTS programs;
CREATE TABLE programs (
id int primary key,
description varchar(64)
);

Done.
Done.


[]

Then I insert all the possible program codes from the inspections table

In [23]:
%%sql
INSERT INTO programs (id)
SELECT DISTINCT CAST("PROGRAM_CATEGORY" as int) FROM inspections;

25 rows affected.


[]

I then update the value of the program description to match the description in the inspections table

In [24]:
%%sql
UPDATE programs as p
SET description = i."PROGRAM_CATEGORY_DESCRIPTION"
FROM inspections as i
WHERE id = CAST(i."PROGRAM_CATEGORY" as int)

25 rows affected.


[]

Let's take a look at our new table:

In [19]:
%%sql
SELECT * FROM programs

25 rows affected.


id,description
402,Childcare
403,Bakery
404,Institutional Kitchen
405,Food Processor
406,Food Service Establishment
407,Limited Menu
408,School
409,Non School
410,NOT FOR PROFIT KITCHEN PERMIT
411,None


For further queries, it is probably good to change the data type of program description in table inspections to match the data type used in our new normalized table

In [20]:
%%sql
ALTER TABLE inspections
ALTER COLUMN "PROGRAM_CATEGORY" SET DATA TYPE int USING "PROGRAM_CATEGORY"::integer

Done.


[]

Re-insert foreign key references to the original inspections table

In [26]:
%%sql
ALTER TABLE inspections
ADD FOREIGN KEY ("PROGRAM_CATEGORY") REFERENCES programs (id)

Done.


[]